In [3]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [4]:
RANDOM_SEED = 42
NUM_CLASSES = 4
TIME_STEPS = 16
DIMENSION = 2

In [5]:
dataset = "model/point_history_classifier/point_history.csv"
model_save_path = "model/point_history_classifier/point_history.hdf5"


In [6]:
X_dataset = np.loadtxt(dataset, delimiter=",", dtype="float32", usecols=list(range(1, (TIME_STEPS * DIMENSION) + 1)))
Y_dataset = np.loadtxt(dataset, delimiter=",", dtype="int32", usecols=(0))

X_train, X_test, Y_train, Y_test = train_test_split(X_dataset, Y_dataset, train_size=0.8, random_state=RANDOM_SEED)


In [7]:
model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION, )),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

2023-05-16 10:06:34.150577: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 24)                792       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                250       
                                                                 
 dense_2 (Dense)             (None, 4)                 44        
                                                                 
Total params: 1,086
Trainable params: 1,086
Non-trainable params: 0
_________________________________________________________________


In [9]:
callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False
)

abort_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [11]:
model.fit(
    X_train,
    Y_train,
    epochs=1000,
    batch_size=128,
    validation_data=[X_test, Y_test],
    callbacks=[callback, abort_callback]
)

Epoch 1/1000
34/34 [==============================] - ETA: 0s - loss: 1.3726 - accuracy: 0.3775
Epoch 1: saving model to model/point_history_classifier/point_history.hdf5
34/34 [==============================] - 4s 40ms/step - loss: 1.3726 - accuracy: 0.3775 - val_loss: 1.3582 - val_accuracy: 0.5613
Epoch 2/1000
26/34 [=====================>........] - ETA: 0s - loss: 1.3460 - accuracy: 0.5042
Epoch 2: saving model to model/point_history_classifier/point_history.hdf5
34/34 [==============================] - 1s 16ms/step - loss: 1.3435 - accuracy: 0.5040 - val_loss: 1.3274 - val_accuracy: 0.5651
Epoch 3/1000
32/34 [===========================>..] - ETA: 0s - loss: 1.3142 - accuracy: 0.5115
Epoch 3: saving model to model/point_history_classifier/point_history.hdf5
34/34 [==============================] - 1s 26ms/step - loss: 1.3134 - accuracy: 0.5092 - val_loss: 1.2887 - val_accuracy: 0.5660
Epoch 4/1000
33/34 [============================>.] - ETA: 0s - loss: 1.2725 - accuracy: 0.5339
E

In [12]:
model = tf.keras.models.load_model(model_save_path)
model.save(model_save_path, include_optimizer=False)

In [13]:
tfl_path = "model/keypoint_classifier/point_history_classifier.tflite"

convertio = tf.lite.TFLiteConverter.from_keras_model(model=model)
convertio.optimizations = [tf.lite.Optimize.DEFAULT]
tfl_quantized_model = convertio.convert()

open(tfl_path, "wb").write(tfl_quantized_model)

2023-05-16 10:10:23.833686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,32]
	 [[{{node input_1}}]]
2023-05-16 10:10:24.015005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]
2023-05-16 10:10:24.058316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,32]
	 [[{{node input_1}}]]
2023-05-16 10:1

INFO:tensorflow:Assets written to: /tmp/tmp6u5g4k1w/assets


INFO:tensorflow:Assets written to: /tmp/tmp6u5g4k1w/assets
2023-05-16 10:10:27.128009: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-16 10:10:27.128097: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-16 10:10:27.133803: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp6u5g4k1w
2023-05-16 10:10:27.137916: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-16 10:10:27.137978: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp6u5g4k1w
2023-05-16 10:10:27.150387: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-05-16 10:10:27.166986: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is n

6432